# Process dump via comsvcs DLL
Detects process memory dump via comsvcs.dll and rundll32

## Rule Content
```
- title: Process dump via comsvcs DLL
  id: 09e6d5c0-05b8-4ff8-9eeb-043046ec774c
  status: experimental
  description: Detects process memory dump via comsvcs.dll and rundll32
  references:
  - https://modexp.wordpress.com/2019/08/30/minidumpwritedump-via-com-services-dll/
  - https://twitter.com/SBousseaden/status/1167417096374050817
  author: Modexp (idea)
  date: 2019/09/02
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    rundll_image:
      Image: '*\rundll32.exe'
    rundll_ofn:
      OriginalFileName: RUNDLL32.EXE
    selection:
      CommandLine:
      - '*comsvcs*MiniDump*full*'
      - '*comsvcs*MiniDumpW*full*'
    condition: (rundll_image or rundll_ofn) and selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.credential_access
  - attack.t1003
  falsepositives:
  - unknown
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\rundll32.exe OR file_name_original:"RUNDLL32.EXE") AND process_command_line.keyword:(*comsvcs*MiniDump*full* OR *comsvcs*MiniDumpW*full*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()